In [8]:
import numpy as np

from centrex_tlf import states, hamiltonian

In [33]:
Jmin = None
Jmax = None

nuclear_spins = states.TlFNuclearSpins()
constants = hamiltonian.XConstants()

rtol = None
stol = 1e-3

E = np.array([0, 0, 0])
B = np.array([0, 0, 1e-5])
# smaller than 1e-5 some X states are degenerate again and states.find_exact_states will
# find duplicate states

In [10]:
X_select = states.QuantumSelector(J=list(range(9)))
X_states_approx = states.generate_coupled_states_X(X_select)

In [11]:
_Jmin = min([gs.J for gs in X_states_approx]) if Jmin is None else Jmin
_Jmax = max([gs.J for gs in X_states_approx]) if Jmax is None else Jmax

In [13]:
QN = states.generate_uncoupled_states_ground(
    Js=np.arange(_Jmin, _Jmax + 1), nuclear_spins=nuclear_spins
)
QNc = states.generate_coupled_states_ground(
    Js=np.arange(_Jmin, _Jmax + 1), nuclear_spins=nuclear_spins
)

In [14]:
H_X_uc = hamiltonian.generate_uncoupled_hamiltonian_X(QN, constants=constants)
H_X_uc_func = hamiltonian.generate_uncoupled_hamiltonian_X_function(H_X_uc)

In [15]:
S_transform = hamiltonian.generate_transform_matrix(QN, QNc)

In [35]:
H_X = S_transform.conj().T @ H_X_uc_func(E, B) @ S_transform
if rtol:
    H_X[np.abs(H_X) < np.abs(H_X).max() * rtol] = 0

In [36]:
H_diagonalized = hamiltonian.generate_diagonalized_hamiltonian(
    H_X, keep_order=True, return_V_ref=True, rtol=rtol
)
QN_diag = hamiltonian.matrix_to_states(H_diagonalized.V, list(QNc))

In [37]:
ground_states = states.find_exact_states(
    [1 * gs for gs in X_states_approx],
    list(QNc),
    QN_diag,
    V=H_diagonalized.V,
    # V_ref=H_diagonalized.V_ref_,
)
ground_states = [gs.remove_small_components(stol) for gs in ground_states]

H_X_red = hamiltonian.reduced_basis_hamiltonian(
    QN_diag, H_diagonalized.H, ground_states
)

In [38]:
H_X_red = hamiltonian.reduced_hamiltonian.ReducedHamiltonian(
    H=H_X_red, V=H_diagonalized.V, QN_basis=ground_states, QN_construct=list(QNc)
)